# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [3]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 62 (delta 6), reused 59 (delta 4), pack-reused 0
Receiving objects: 100% (62/62), 90.16 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (6/6), done.


Install all required dependencies:

In [3]:
!pip install -r ../requirements.txt

  Using cached fire-0.6.0-py2.py3-none-any.whl
  Using cached simple_parsing-0.1.6-py3-none-any.whl.metadata (7.3 kB)
  Using cached mistral_common-1.4.2-py3-none-any.whl.metadata (4.4 kB)
  Using cached safetensors-0.4.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached torch-2.2.0-cp312-none-macosx_11_0_arm64.whl.metadata (25 kB)
ERROR: Could not find a version that satisfies the requirement triton==2.2 (from versions: none)
ERROR: No matching distribution found for triton==2.2


## Model download

In [ ]:
!pip install huggingface_hub

In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

In [5]:
# Alternatively, you can download the model from mistral

# !wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-05-24 18:50:25--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  40.5MB/s    in 6m 3s   

2024-05-24 18:56:29 (38.1 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [7]:
# !DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [2]:
#!ls /content/mistral_models
!ls ../../mistral-7B-v0.3

consolidated.safetensors params.json              tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [5]:
%cd ../content/

/Users/luweiying/Documents/mistral-finetune/content


/Users/luweiying/Documents/mistral-finetune/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
# make a new directory called data
!mkdir -p data

In [8]:
# navigate to this data directory
%cd data

/Users/luweiying/Documents/mistral-finetune/content/data


In [16]:
import pandas as pd

# df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')
df = pd.read_parquet('test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [17]:
df

,prompt,prompt_id,messages
0,"This story begins with an end. In March 1991, ...",5ee2fbb48ef35593b81444d7aec405bb4f152abbe80f7b...,[{'content': 'This story begins with an end. I...
1,Explain how the invention and widespread use o...,fc6aae406cd26c79db4d35dd32bcbd8ee0f1493a0096b5...,[{'content': 'Explain how the invention and wi...
2,Read the passage below and answer the question...,44a13514d9cd363d85479ff25e5837c60c5f90815428c2...,[{'content': 'Read the passage below and answe...
3,Explain the influence of culture on attitudes ...,c0c7f2a08bd4dc84bc527d774b1fe411eefa7bcdb847b5...,[{'content': 'Explain the influence of culture...
4,Can you provide data on the employment rates i...,b26cb026578e891c3ccd0cf075da6cffaa05df05412aa0...,[{'content': 'Can you provide data on the empl...
...,...,...,...
28299,How have the TIMI trials contributed to our un...,42cf5424e1f8a3ddf7670dd9273620cfeb42c9d2c3d746...,[{'content': 'How have the TIMI trials contrib...
28300,Write step-by-step instructions for making a h...,da5a99e17e7be7a4ee51e8f54709a0347a6e82afa8cf1a...,[{'content': 'Write step-by-step instructions ...
28301,"Using Unity, create a puzzle game that allows ...",597c69c1b58ba7d049ef405ca37045a8c3b7de6898e7e4...,"[{'content': 'Using Unity, create a puzzle gam..."
28302,Please research and find a charity organizatio...,dfe500129711acbbd609fe4b7f1f19414cecd6e041c403...,[{'content': 'Please research and find a chari...


In [18]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [19]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [21]:
!ls 

test_gen-00000-of-00001-3d4cd8309148a71f.parquet
ultrachat_chunk_eval.jsonl
ultrachat_chunk_train.jsonl


In [34]:
# navigate to the mistral-finetune directory
%cd /Users/luweiying/Documents/mistral-finetune

/Users/luweiying/Documents/mistral-finetune


/Users/luweiying/Documents/mistral-finetune/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [39]:
pwd

'/Users/luweiying/Documents/mistral-finetune'

In [38]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that don't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/luweiying/Documents/mistral-finetune/utils/reformat_data.py", line 88, in <module>
    reformat_jsonl(args.file)
  File "/Users/luweiying/Documents/mistral-finetune/utils/reformat_data.py", line 13, in reformat_jsonl
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
         ^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/data/ultrachat_chunk_train.jsonl'


In [40]:
# eval data looks all good
!python -m utils.reformat_data /Users/luweiying/Documents/mistral-finetune/content/data/ultrachat_chunk_eval.jsonl

In [49]:
pwd

'/Users/luweiying/Documents/mistral-finetune'

In [50]:
import json
import random

def sample_jsonl_file(input_file, output_file, num_samples=10):
    # Load all lines from the input JSONL file
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # Randomly sample the specified number of lines
    sampled_lines = random.sample(lines, min(num_samples, len(lines)))
    
    # Write the sampled lines to the new JSONL file
    with open(output_file, 'w') as out_f:
        for line in sampled_lines:
            out_f.write(line)

# Path to your input and output JSONL files
input_file = 'content/data/project_skill_question.jsonl'
output_file = 'content/data/sampled_project_skill_question.jsonl'

# Get 10 random samples and save to a new file
sample_jsonl_file(input_file, output_file, 10)

In [80]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating /Users/luweiying/Documents/mistral-finetune/content/data/sampled_project_skill_question.jsonl ...

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 614.96it/s]
1it [00:00, 59.89it/s]
No errors! Data is correctly formatted!
Stats for /Users/luweiying/Documents/mistral-finetune/content/data/sampled_project_skill_question.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:14:32",
        "data_tokens": 9266,
        "train_tokens": 26214400,
        "epochs": "2829.10",
        "max_steps": 100,
        "data_tokens_per_dataset": {
            "/Users/luweiying/Documents/mistral-finetune/content/data/sampled_project_skill_question.jsonl": "9266.0"
        },
        "train_tokens_per_dataset": {
            "/Users/luweiying/Documents/mistral-finetune/content/data/sampled_project_skill_question.jsonl": "26214400.0"
        },
        "epochs_per_dataset": {
            "/Users/luweiying/Documents/mistral-finetune/content/

## Start training

In [52]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [20]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [21]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [94]:
# start training

!torchrun --nproc-per-node=1 --nnodes=1 -m train example/7Bcpu.yaml

W0918 18:28:40.546000 8699023168 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/Users/luweiying/Documents/mistral-finetune/content/data/sampled_project_skill_question.jsonl', eval_instruct_data='', instruct=InstructArgs(shuffle=True, dynamic_chunk_fn_call=True)), model_id_or_path='/Users/luweiying/Documents/mistral-7B-v0.3', run_dir='', optim=OptimArgs(lr=6e-05, weight_decay=0.1, pct_start=0.05), seed=0, num_microbatches=1, seq_len=32768, batch_size=1, max_norm=1.0, max_steps=100, log_freq=1, ckpt_freq=100, save_adapters=True, no_ckpt=False, num_ckpt_keep=3, eval_freq=100, no_eval=True, checkpoint=True, world_size=1, wandb=WandbArgs(project='None', offline=True, key='None', run_name='None'), mlflow=MLFlowArgs(tracking_uri=None, experiment_name=None), lora=LoraArgs(enable=True, rank=64, dropout=0.0, scaling=2.0))
2024-09-18 18:28:41 (PST) - 0

## Inference

In [24]:
!pip install mistral_inference

In [25]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of computer science that enables machines to learn and improve from experience without being explicitly programmed. Machine learning algorithms can learn from data and make predictions or decisions based
